In [ ]:
import os
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
from pathlib import Path
import re
from skimage import measure
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
import matplotlib as mpl
import tqdm
import tensorflow as tf
from tensorflow_examples.models.pix2pix import pix2pix
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import normalize
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda


In [ ]:
images_path = "/Users/ciro/Documents/dataset"
masks_path = "/Users/ciro/Documents/dataset"

In [ ]:
#Capture training image info as a list
images_paths = []
masks_paths = [] 
limit=50
def get_images (grade):
    idx=0
    grade_dir = os.path.join(images_path, grade)
    for imgname in os.listdir(grade_dir):
        if(idx<limit):
            if(imgname!='.DS_Store'):
                file_name = f"{imgname}_flair.nii"
                file_path = os.path.join(grade_dir, imgname, file_name)
                images_paths.append(file_path)
                idx+=1
          
     
    

def get_masks (grade):
   idx=0
   grade_dir = os.path.join(masks_path, grade)
   for imgname in os.listdir(grade_dir):
        if(idx<limit):
            if(imgname!='.DS_Store'):
                file_name = f"{imgname}_seg.nii"
                file_path = os.path.join(grade_dir, imgname, file_name)
                masks_paths.append(file_path)
                idx+=1
            
          
    
    
get_images('HGG')
get_images('LGG')
get_masks('HGG')
get_masks('LGG')

In [ ]:
len(images_paths)
#images_paths.pop(0)
#images_paths.sort()

In [ ]:
masks_paths.sort()
#masks_paths.pop(0)

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
import os

# Define the slicing function
def load_and_slice_image(img_path, mask_path, target_shape=(128, 128)):
    img = nib.load(img_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()
    
    slices_img = []
    slices_mask = []
    
    for i in range(img.shape[2]):
        slice_img = resize(img[:, :, i], target_shape, mode='constant', preserve_range=True)
        slice_mask = resize(mask[:, :, i], target_shape, mode='constant', preserve_range=True, order=0)
        
        slices_img.append(slice_img)
        slices_mask.append(slice_mask)
    
    return np.array(slices_img), np.array(slices_mask)



# Corrected code to get images and masks paths
images_paths = []
masks_paths = [] 

def get_images(grade):
    idx=0
    grade_dir = os.path.join(images_path, grade)
    for imgname in os.listdir(grade_dir):
        if(idx<limit):  
            if(imgname!='.DS_Store'):   
                file_name = f"{imgname}_flair.nii"
                file_path = os.path.join(grade_dir, imgname, file_name)
                images_paths.append(file_path)
                idx+=1

def get_masks(grade):
    idx=0
    grade_dir = os.path.join(masks_path, grade)
    for imgname in os.listdir(grade_dir):
        if(idx<limit): 
            if(imgname!='.DS_Store'):
                file_name = f"{imgname}_seg.nii"
                file_path = os.path.join(grade_dir, imgname, file_name)
                masks_paths.append(file_path)
                idx+=1
            
get_images('HGG')
get_images('LGG')
get_masks('HGG')
get_masks('LGG')

# Load and slice images and masks
all_slices_img = []
all_slices_mask = []

for img_path, mask_path in zip(images_paths, masks_paths):
    slices_img, slices_mask = load_and_slice_image(img_path, mask_path)
    all_slices_img.extend(slices_img)
    all_slices_mask.extend(slices_mask)

all_slices_img = np.expand_dims(np.array(all_slices_img), axis=-1)
all_slices_mask = np.expand_dims(np.array(all_slices_mask), axis=-1)

print("Total slices of images:", all_slices_img.shape)
print("Total slices of masks:", all_slices_mask.shape)


In [ ]:
import nibabel as nib
import numpy as np

def load_images(images_paths):
    images = []
    
    for img_path in (images_paths):
        flair_img = nib.load(img_path).get_fdata()
         
        
        images.append(flair_img)
         
    
    return np.array(images) 

def load_masks(masks_paths):
     
    masks = []
    
    for  mask_path in ( masks_paths):
         
        seg_img = nib.load(mask_path).get_fdata()
        
         
        masks.append(seg_img)
    
    return  np.array(masks)

images=load_images(images_paths)
masks=load_masks(masks_paths)
 

print("Images shape:", images.shape)
print("Masks shape:", masks.shape)


In [ ]:
from skimage.transform import resize

def preprocess_data(images, masks, target_shape=(128, 128)):
    preprocessed_images = []
    preprocessed_masks = []
    
    for img, mask in zip(images, masks):
        for i in range(img.shape[2]):  # Iterate through slices
            slice_img = resize(img[:, :, i], target_shape, mode='constant', preserve_range=True)
            slice_mask = resize(mask[:, :, i], target_shape, mode='constant', preserve_range=True, order=0)
            
            preprocessed_images.append(slice_img)
            preprocessed_masks.append(slice_mask)
    
    return np.expand_dims(np.array(preprocessed_images), axis=-1), np.expand_dims(np.array(preprocessed_masks), axis=-1)

target_shape = (128, 128)
preprocessed_images, preprocessed_masks = preprocess_data(images, masks, target_shape)

print("Preprocessed images shape:", preprocessed_images.shape)
print("Preprocessed masks shape:", preprocessed_masks.shape)


In [ ]:
import matplotlib.pyplot as plt

def visualize_data(images, masks, num_samples=3):
    plt.figure(figsize=(10, num_samples * 5))
    
    for i in range(num_samples):
        idx = np.random.randint(0, len(images))
        plt.subplot(num_samples, 2, 2 * i + 1)
        plt.imshow(images[idx].squeeze(), cmap='gray')
        plt.title('FLAIR Image')
        
        plt.subplot(num_samples, 2, 2 * i + 2)
        plt.imshow(masks[idx].squeeze(), cmap='gray')
        plt.title('Segmentation Mask')
    
    plt.show()

visualize_data(preprocessed_images, preprocessed_masks)
